In [ ]:
%matplotlib inline
from pylab import *
import os
import sys
from importlib import reload
import glob
import pickle

# Specific science modules
import healpy as hp
import numpy as np

# Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm
import qubic
from qubic import SpectroImLib as si
from qubic import QubicSkySim as qss
from qubic import NamasterLib as nam
from qubic import camb_interface as qc
from qubic import fibtools as ft


rc('figure', figsize=(13, 10))
rc('font', size=12)

In [ ]:
#### This function reads the maps
def read_files_mc(dirmap,file_rnd_string, verbose=False):
    m = np.array(FitsArray(glob.glob(dirmap+'*_maps_recon_*'+file_rnd_string+'.fits')[0]))
    c = np.array(FitsArray(glob.glob(dirmap+'*_maps_convolved_*'+file_rnd_string+'.fits')[0]))
    cov = np.array(FitsArray(glob.glob(dirmap+'*_maps_coverage_*'+file_rnd_string+'.fits')[0]))
    with open(glob.glob(dirmap+'*_dictionary_'+file_rnd_string+'.pickle')[0], 'rb') as handle: d = pickle.load(handle)
    with open(glob.glob(dirmap+'*_input_cell_'+file_rnd_string+'.pickle')[0], 'rb') as handle: icl = pickle.load(handle)
    filetype = glob.glob(dirmap+'*_maps_recon_*'+file_rnd_string+'.fits')[0].split('/')[-1]
    if verbose: print('read {}'.format(filetype))
    return m, c, cov, d, icl, filetype
    


In [ ]:
# #### nptg=1000 - tol = 1e-5 rom MacQUBIC
# nptg = 1000
# dirmaps = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Maps/'
# file_rnd_string = 'zqGAikhUO0'

# #### nptg=3000 - tol = 1e-5 rom MacQUBIC
# nptg = 3000
# dirmaps = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Maps/'
# file_rnd_string = 'zR2hSgphTA'

#### nptg=3000 - tol = 1e-5 rom NERSC
# nptg = 3000
# dirmaps = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB_NERSC/Maps/'
# file_rnd_string = 'UDrLEmD7gc'

# #### nptg=10000 - tol = 1e-5 rom NERSC
nptg = 10000
dirmaps = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB_NERSC/Maps/'
file_rnd_string = 'PFdeS3NvBg'

m, c, cov, d, icl, ftype = read_files_mc(dirmaps, file_rnd_string, verbose=True)

### Center of the QUBIC Field
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])

### Input Th Spectra
input_cell = icl.copy()
ellth= np.arange(icl.shape[0])

### Check the coverages
figure()
hp.gnomview(cov, rot=center, reso=15, title='Coverage 0', sub=(1,3,1))

### Check convolved maps
figure()
stn = ['I', 'Q', 'U']
for s in range(3):
    hp.gnomview(c[0,:,s], rot=center, reso=15, title='Convolved '+stn[s], sub=(3,3,1+s))    
    
### Check Reconstructed maps
figure()
stn = ['I', 'Q', 'U']
for s in range(3):
    hp.gnomview(m[0,:,s], rot=center, reso=15, title='Reconstructed '+stn[s], sub=(3,3,1+s))    


In [ ]:
seenpix = cov > (0.1*np.max(cov))

covnorm = cov / np.max(cov)

nbins = 30
xx, yyI, dx, dyI, _ = ft.profile(np.sqrt(1./covnorm[seenpix]), m[0,seenpix,0]-c[0,seenpix,0], nbins=nbins, plot=False)
xx, yyQ, dx, dyQ, _ = ft.profile(np.sqrt(1./covnorm[seenpix]), m[0,seenpix,1]-c[0,seenpix,1], nbins=nbins, plot=False)
xx, yyU, dx, dyU, _ = ft.profile(np.sqrt(1./covnorm[seenpix]), m[0,seenpix,2]-c[0,seenpix,2], nbins=nbins, plot=False)
avg = (dyI+dyQ/np.sqrt(2)+dyU/np.sqrt(2))/3
plot(xx,avg, label='QUBIC MC Average IQU')
xlabel('1./sqrt(cov normed)')
ylabel('RMS Noise per pixel')

plot(xx,xx/xx[0]*avg[0])

title('NPTG={} File='.format(nptg)+ftype)
ylim(0.1,10)
grid()
legend()



In [ ]:
### All together
all_nptg = []
all_dirmaps = []
all_file_rnd_string = []

# #### nptg=1000 - tol = 1e-5 rom MacQUBIC
all_nptg.append(1000)
all_dirmaps.append('/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Maps/')
all_file_rnd_string.append('zqGAikhUO0')

# #### nptg=3000 - tol = 1e-5 rom MacQUBIC
all_nptg.append(3000)
all_dirmaps.append('/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB/Maps/')
all_file_rnd_string.append('zR2hSgphTA')

#### nptg=3000 - tol = 1e-5 rom NERSC
all_nptg.append(3000)
all_dirmaps.append('/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB_NERSC/Maps/')
all_file_rnd_string.append('UDrLEmD7gc')

# #### nptg=10000 - tol = 1e-5 rom NERSC
all_nptg.append(10000)
all_dirmaps.append('/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/SimsPureCMB_NERSC/Maps/')
all_file_rnd_string.append('PFdeS3NvBg')





for i in range(len(all_nptg)):
    m, c, cov, d, icl, ftype = read_files_mc(all_dirmaps[i], all_file_rnd_string[i], verbose=True)

    seenpix = cov > (0.1*np.max(cov))
    covnorm = cov / np.max(cov)

    nbins = 30
    xx, yyI, dx, dyI, _ = ft.profile(np.sqrt(1./covnorm[seenpix]), m[0,seenpix,0]-c[0,seenpix,0], nbins=nbins, plot=False)
    xx, yyQ, dx, dyQ, _ = ft.profile(np.sqrt(1./covnorm[seenpix]), m[0,seenpix,1]-c[0,seenpix,1], nbins=nbins, plot=False)
    xx, yyU, dx, dyU, _ = ft.profile(np.sqrt(1./covnorm[seenpix]), m[0,seenpix,2]-c[0,seenpix,2], nbins=nbins, plot=False)
    avg = (dyI+dyQ/np.sqrt(2)+dyU/np.sqrt(2))/3
    plot(xx,avg, label='QUBIC MC Average IQU Nptg={}'.format(all_nptg[i]))

xlabel('1./sqrt(cov normed)')
ylabel('RMS Noise per pixel')

plot(xx,xx*avg[0]/xx[0],'k--',label='ideal')
ylim(0,8)
#yscale('log')
grid()
legend()
                   
                   
